Given a string num that contains only digits and an integer target, return all possibilities to insert the binary operators '+', '-', and/or '*' between the digits of num so that the resultant expression evaluates to the target value.

Note that operands in the returned expressions should not contain leading zeros.

 

Example 1:

Input: num = "123", target = 6
Output: ["1*2*3","1+2+3"]
Explanation: Both "1*2*3" and "1+2+3" evaluate to 6.
Example 2:

Input: num = "232", target = 8
Output: ["2*3+2","2+3*2"]
Explanation: Both "2*3+2" and "2+3*2" evaluate to 8.
Example 3:

Input: num = "3456237490", target = 9191
Output: []
Explanation: There are no expressions that can be created from "3456237490" to evaluate to 9191.
 

Constraints:

1 <= num.length <= 10
num consists of only digits.
-231 <= target <= 231 - 1

In [32]:
class Solution:
    def __init__(self):
        self.res: list[str] = []

    def addOperators(self, num: str, target: int) -> list[str]:
        self.addOperators_rec(num, 1, target, int(num[0]), num[0], len(num))
        return self.res 

    def addOperators_rec(self, num, ind, target, cur_val, cur_exp, n):
        # return logic.
        if ind == n:
            if cur_val == target:
                self.res.append(cur_exp)
            return
            
        # Gonna do the + operation.
        last_digit = int(num[ind])
        cur_val += last_digit
        self.addOperators_rec(num, ind+1, target, cur_val, cur_exp +"+"+str(last_digit), n)
        cur_val -= last_digit
        
        # Gonna do the - operation.
        cur_val -= last_digit
        self.addOperators_rec(num, ind+1, target, cur_val, cur_exp+"-"+str(last_digit), n)
        cur_val += last_digit
        
        # Gonna do the * operation.
        cur_val *= last_digit
        self.addOperators_rec(num, ind+1, target, cur_val, cur_exp+"*"+str(last_digit), n)
        cur_val //= last_digit
        

In [33]:
Solution().addOperators(num = "123", target = 6)

['1+2+3', '1*2*3']

In [34]:
Solution().addOperators(num = "232", target = 8)

['2*3+2']

In [35]:
Solution().addOperators(num = "3456237490", target = 9191)

ZeroDivisionError: integer division or modulo by zero

# problems with the above approach:
- we are considering only one digit at a time, but we can take any number of digits.
- if first digit is zero - do nothing.
- if multiplication, you have to do it according to precedence -- 2+3*2 == 2 + (3*2) -- we cant do this is in our logic.

In [ ]:
class Solution:
    def addOperators(self, num: str, target: int) -> list[str]:
        res = []

        def backtrack(index, prev_operand, cur_val, expr):
            if index == len(num):
                if cur_val == target:
                    res.append(expr)
                return

            for i in range(index, len(num)):
                # Skip numbers with leading 0
                if i != index and num[index] == '0':
                    break

                curr_str = num[index:i+1]
                curr_num = int(curr_str)

                if index == 0:
                    # First number, no operator
                    backtrack(i+1, curr_num, curr_num, curr_str)
                else:
                    backtrack(i+1, curr_num, cur_val + curr_num, expr + "+" + curr_str)
                    backtrack(i+1, -curr_num, cur_val - curr_num, expr + "-" + curr_str)
                    # For '*', use prev_operand to handle precedence.
                    # we have to specially handle the mulitplication to make the, precedence work.
                    backtrack(i+1, prev_operand * curr_num,
                              cur_val - prev_operand + (prev_operand * curr_num),
                              expr + "*" + curr_str)

        backtrack(0, 0, 0, "")
        return res




Expression so far: "2+3"

cur_val = 5
prev_operand = 3
curr = 2

Fix it:
new_val = cur_val - prev_operand + (prev_operand * curr)
        = 5 - 3 + (3 * 2)
        = 2 + 6
        = 8 ✅




## ✅ Time Complexity: **O(4ⁿ)**

Where:

* `n = len(num)`

### Why O(4ⁿ)?

For each position between digits (there are `n-1` such positions), we have up to **3 operator choices**: `'+'`, `'-'`, or `'*'`.
Additionally, we can also choose to **not split** digits — for example, `"123"` → we can treat it as:

* `"1+2+3"`
* `"12+3"`
* `"1+23"`
* `"123"`

So at each step, we have:

* 1 choice to **extend** the current operand
* 3 choices to **insert an operator** (if we split)

Hence, total combinations can go up to `4^n` in the worst case (though not all are valid).

---

## 🧠 Example (n = 4):

For `num = "1234"`, you could have expressions like:

* `"1+2+3+4"`
* `"12-3*4"`
* `"1*23-4"`
* and so on...

---

## ✅ Space Complexity: **O(n)**

* We use **recursion**, and the recursion depth is at most `n` (one call per digit).
* Expression strings are built along the way — each string can be at most `2n-1` in length (`n` digits + `n-1` operators), so it's **linear**.

If we count all valid expressions in the result list, then:

* Worst case result list could contain **up to O(4ⁿ)** strings → in practice, it's much fewer due to invalid paths and pruning.

---

## 🧾 Summary

| Complexity         | Value       | Explanation                                |
| ------------------ | ----------- | ------------------------------------------ |
| Time               | O(4ⁿ)       | Try every split + 3 ops (`+`, `-`, `*`)    |
| Space (call stack) | O(n)        | Max recursion depth                        |
| Space (results)    | O(4ⁿ) worst | All valid expressions (pruned in practice) |
